In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
from glob import glob
import json
import csv
import os

import cv2
import numpy as np

In [4]:
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [204]:
from keras.optimizers import Adam
from keras.layers import Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [7]:
from utils import read_samples
from utils import ImageGenerator
from utils import CenterImageGenerator

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
ROOT_DIR = os.path.join(os.getcwd(), '../')

In [10]:
DATA_DIR = os.path.join(ROOT_DIR, 'data')

In [11]:
samples = read_samples(DATA_DIR)

Loaded 13305 samples.


In [12]:
def append_path(line):
    line[0] = os.path.join(ROOT_DIR, line[0])
    line[1] = os.path.join(ROOT_DIR, line[1])
    line[2] = os.path.join(ROOT_DIR, line[2])
    return line

In [13]:
samples = [append_path(line) for line in samples]

In [14]:
train_set, test_set = train_test_split(samples, test_size=0.2)
train_set, valid_set = train_test_split(train_set, test_size=0.2)

In [47]:
train_iterator = CenterImageGenerator(train_set, batch_size=128, flip=True, gray=True)
valid_iterator = CenterImageGenerator(valid_set, batch_size=128, flip=True, gray=True)

In [201]:
test_iterator = CenterImageGenerator(test_set, batch_size=128, gray=True)

In [220]:
def lenet_model(input_shape):
    _input = Input(shape=input_shape)
    norm = Lambda(lambda x: x / 255 - 0.5)(_input)
    conv1 = Conv2D(filters=6, kernel_size=(3, 3), strides=(1, 1), 
                   padding='valid', activation='relu')(norm)
    pool1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv1)
    conv2 = Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), 
                   padding='valid', activation='relu')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv2)
    #conv3 = Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), 
    #                padding='valid', activation='relu')(pool2)
    #pool3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv3)
    flatten = Flatten()(pool2)
    fc1 = Dense(120, activation='relu')(flatten)
    fc2 = Dense(84, activation='linear')(fc1)
    output = Dense(1, activation='linear')(fc2)
    model = Model(inputs=[_input], outputs=[output])
    return model

In [221]:
adam = Adam(0.0009)
model = lenet_model((80, 320, 1))
model.compile(loss='mse', optimizer=adam)

In [222]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, 
                               patience=1, verbose=0, mode='auto')
filepath = 'lenet.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, 
                save_weights_only=False, mode='auto', period=1)

In [223]:
model.fit_generator(generator=train_iterator, epochs=10, 
                    validation_data=valid_iterator,
                    steps_per_epoch=len(train_iterator), 
                    validation_steps=len(valid_iterator),
                    callbacks=[early_stopping, checkpoint]
                   )

Epoch 1/10
134/134 [==============================] - 24s - loss: 0.0233 - val_loss: 0.0284
Epoch 2/10
134/134 [==============================] - 23s - loss: 0.0012 - val_loss: 0.0273
Epoch 3/10
134/134 [==============================] - 23s - loss: 0.0013 - val_loss: 0.0226
Epoch 4/10
134/134 [==============================] - 22s - loss: 0.0019 - val_loss: 0.0279
Epoch 5/10
134/134 [==============================] - 23s - loss: 0.0019 - val_loss: 0.0309


In [224]:
results = model.evaluate_generator(generator=test_iterator, steps=len(test_iterator))

In [225]:
results

0.016926652751863003

In [138]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 80, 320, 1)        0         
_________________________________________________________________
lambda_19 (Lambda)           (None, 80, 320, 1)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 76, 316, 8)        208       
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 38, 158, 8)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 34, 154, 16)       3216      
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 17, 77, 16)        0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 20944)             0         
__________